## LangChain Basics

When you want to interact with OpenAI’s models via LangChain, you can use the `ChatOpenAI` class. This class serves as a convenient wrapper around the OpenAI ChatCompletion API, allowing you to structure different message types (e.g., system, human, or AI messages) and providing methods—such as invoke()—for sending prompts and receiving responses from the model.

For more details, see the official documentation:
https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html


In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

load_dotenv()
openai_api_key = os.getenv('OPEN_AI_API_KEY')

model = ChatOpenAI(model="gpt-4o-mini")

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages).content


'Ciao!'

## Tools

Tools in LangChain are one of its more powerful features. They enable you to connect Python functions to a language model. By integrating tools, an LLM can execute code you write to make API calls, query databases, or perform other tasks.

Of course, getting the LLM to pass the right arguments is the major design concern here. LangChain supports multiple approaches for specifying tool interfaces. For simpler scenarios, you can use a decorator and rely on a function’s docstring to guide the LLM. For more complex cases, you can define a more explicit schema using tools like pydantic.

Lets look at setting up a simple tool using a decorator and a docstring.

In [63]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

polygon_api_key = os.getenv('POLYGON_API_KEY')

@tool
def get_apple_close_on_date(date:str) -> int:
    """Gets the closing stock price for Apple on the latest trading day.

    Args:
        date: str - the date formatted as YYYY-MM-DD

    Returns:
        Last close price
    """
    url = "https://api.polygon.io/v1/open-close/AAPL/"+date
    params = {
        "adjusted": "true",
        "apiKey": polygon_api_key,
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()  
    response = response.json()
    close = response['close']
    
    return close

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools = [get_apple_close_on_date]
model = model.bind_tools(tools)

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)


query='What was the closing share price of Apple on 2024-12-10?'
agent_executor.invoke({"input": query})

{'input': 'What was the closing share price of Apple on 2024-12-10?',
 'output': 'The closing share price of Apple on December 10, 2024, was $247.77.'}

Here we can see our agent took care of calling the tool get_apple_close_on_date which is a function that accepts a date argument. We made it easy and passed it the date in the format it wanted, but as you can see below, if we pass a date that would require a change in format for the function to work that can also be accomplished. Now, to my surprise at the time of writing this, I thought this was due to the doc string showing the expected format of the date arg, however if tested without this format it still works. It turns out, the default format of YYYY-MM-DD is what the LLM expects based on its own training data. 

In [62]:
query='What was the closing share price of Apple on December 12th 2024?'
agent_executor.invoke({"input": query})

{'status': 'OK', 'from': '2024-12-12', 'symbol': 'AAPL', 'open': 246.89, 'high': 248.74, 'low': 245.68, 'close': 247.96, 'volume': 30928444.0, 'afterHours': 247.15, 'preMarket': 247.5}


{'input': 'What was the closing share price of Apple on December 12th 2024?',
 'output': 'The closing share price of Apple on December 12th, 2024, was $247.96.'}

## Pydantic Tools

One of the more common solutions for defining tool schemas is to use Pydantic, which is an easy to use library to support schema generation and data validation using Python's type hints. It will automatically validate input data against the defined type hints, for instance if you have type hinted that an arg should be an int, Pydantic will throw an exception if the data passed is not an int. Lets look at this in the contex of a tool setup. 

In [ ]:
@tool
def get_close_on_date(tickers, date):
    """Gets the last close price for per stock ticker for a specific date.
    """
    print(tickers)
    results = []
    for t in tickers:
        url = f"https://api.polygon.io/v1/open-close/{t}/{date}"
        params = {
            "adjusted": "true",
            "apiKey": polygon_api_key,
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()  
        response = response.json()
        results.append(response)
    
    return results

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools = [get_close_on_date]
model = model.bind_tools(tools)

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query='What was the closing share price of APPL on December 12th 2024?'
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `get_close_on_date` with `{'tickers': 'AAPL', 'date': '2024-12-12'}`


AAPL


HTTPError: 404 Client Error: Not Found for url: https://api.polygon.io/v1/open-close/P/2024-12-12?adjusted=true&apiKey=jVLZTAifzAvVZAuLdAgQnMdpgycY64a0

In the example above, we expect the tickers arg to be a list. Now we could probably add that as a type hint in the function, and put the request type in the docstring, and in most cases the LLM will figure this out, however not in all. The best way to ensure that arguments are passed with the correct type is to use Pydantic. Lets look at this example below:

In [ ]:
from pydantic import BaseModel, Field
from typing import List


class get_close_on_date(BaseModel):
    """Gets the last close price for per stock ticker for a specific date."""

    tickers: List[str] = Field(description="A list of stock tickers")
    date: str = Field(description="A date string formatted as YYYY-MM-DD")





def get_close_on_date(tickers, date):
    """Gets the last close price for per stock ticker for a specific date.
    """
    print(tickers)
    results = []
    for t in tickers:
        url = f"https://api.polygon.io/v1/open-close/{t}/{date}"
        params = {
            "adjusted": "true",
            "apiKey": polygon_api_key,
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()  
        response = response.json()
        results.append(response)
    
    return results

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools = [get_close_on_date]
model = model.bind_tools(tools)

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

query='What was the closing share price of APPL on December 12th 2024?'
agent_executor.invoke({"input": query})